# Multi-step joining of monthly Sentinel-2 data with points of interest for training and application of a Random-Forest model

## 0. Introduction

This notebook demonstrates how to use the Geo Engine to process raster and vector data to to train an external Random-Forest model for field-use classification.
The generation of the training data is dnoe in the Geo Engine.
The model is trained using sklearn in this notebook.
Then, the model is applied to Sentinel-2 data queried from the Geo Engine.

### Use-Case

Spatial information of field-use is very important for various applications like yield estimation.
Using Earth Observation (EO) data we can generate large raster maps of crop type or field-use.
In this notebook, we want to generate such a map for an area in the state of North Rhine-Westphalia (NRW), Germany.
To generate such a map, we need ground truth data, e.g. sampling points where the real field-use is known.
Then, we can train a Maschine Learning (ML) modell on the sample points and the corresponding values of the EO data.
The trained model can then be applied to the whole area to generate a map of the field-use.

### Data

In this notebook, we use the following data:

#### EuroCrops (NRW, Germany)

As label data (ground truth) we use the EuroCrops data for NRW, Germany.
It is available under the [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).
The data can be downloaded from [here](https://github.com/maja601/EuroCrops#vectordata_zenodo).
This dataset contains field polygons with different classes e.g. the crop type or the usage of the field.
For the demo presented in this notebook, we convert the field polygons into points and use them as sample-points.

#### Sentinel-2

The Sentinel-2 data we use is available from the Element 84 Sentinel-2 L2A Data Hub.
Using the STAC API, it can be queried for areas and times of interest.
The data is available under the [ESA Data License](https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice).
The Geo Engine provides an [ExternalDataProvider] that acts as a STAC API client that can be used to query the data.

### Workflow of this notebook

This notebook focuses on the extraction of the Sentinel-2 data for the points of interest and the model training :
1. Setup of packages, the Geo Engnie session and the area of interest.
2. Download the Sentinel-2 data for the area of interest and store it as a new (local) dataset.
3. Build the workflow for cloud-free monthly means of Sentinel-2 data and derive the NDVI.
4. Attach the aggregated Sentinel-2 to the points of interest.
5. Train a Random-Forest model to classify the field usage based on the Sentinel-2 data attached to the points.
6. Apply the model to Sentinel-2 data queried from the Geo Engine.


## 1. Setup


This notebook requires the following packages:

In [ ]:
from datetime import datetime
import asyncio
import geopandas as gpd
import geoengine as ge
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
from asyncstdlib.itertools import zip_longest
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier


Connect to a Geo Engine instance. You need to provide user credentials via parameters or environment variables.

In [ ]:
ge.initialize("http://localhost:3030/api")

In [ ]:
session = ge.get_session()
user_id = session.user_id
session

Set the area of interest. It is defined as a bounding box in UTM 32 N (EPSG:32632).
It is locted in NRW, Germany and covers the area between Willingen, Lippstadt and Werl.

In [ ]:
[xmin, ymin, xmax, ymax] = [421395,  5681078, 476201, 5727833]
size_x = xmax - xmin
size_y = ymax - ymin
print(size_x, size_y)
(xmin, ymin, xmax, ymax)

Using the bounding box, a time interval and a resolution, we define the area of interest as a temporal raster space-time cube.
The time interval is defined as a start and end date.
Since the field data is available for the year 2018, we use this year as the time interval.
The Sentinel-2 data has a resolution of 10 m, so we use this resolution for the area of interest as well.

In [ ]:
time_start = datetime(2021, 1, 1)
time_end = datetime(2022, 1, 1)

study_area = ge.api.RasterQueryRectangle(
    spatialBounds=ge.SpatialPartition2D(xmin, ymin, xmax, ymax).to_api_dict(),
    timeInterval=ge.TimeInterval(time_start, time_end).to_api_dict(),
    spatialResolution=ge.SpatialResolution(100.0, 100.0).to_api_dict(),
)
study_area

## 2. Download Sentinel-2 data and store it in the Geo Engine

The Sentinel-2 data is stored in the cloud (AWS S3).
There is a STAC API that provides access to the data. 
To use the data in the Geo Engine, we create workflows that accesses the different Sentinel-2 bands and make them available.
This first kind of workflow only queries the data in the area of interest and stores it as local datasets.
This way, we don't need to download the data every time we use it.

For each band (B02, B03, B04, B08) as well as the scene mask (SCL), we create a workflow using the `sentinel2_band(band_name)` blueprint.
This convenience method creates a workflow that uses the [`GdalSource`](https://docs.geoengine.io/operators/gdalsource.html) to load the Sentinel-2 data.
A source operator like the `GdalSource` takes a `DatasetId` that identifies the data to load.
In this case, the `DatasetId` is provided by an external dataset provider, that resolves band "names" to the information how to load the data from STAC / S3.
Using the `save_as_dataset(study_area)` method, we tell the Geo Engine to store the data as a new dataset.
The new dataset gets a unique `DatasetId` that we can use to load the data in the next step.

In [ ]:
download_tasks = {}
s2_data_prefix = user_id + ":y_sentinel2_nrw_crop_10m_"

for b in ["B02", "B03", "B04", "B08", "SCL"]:
    sentinel2_band_workflow = ge.workflow_builder.blueprints.sentinel2_band(b)
    sentinel2_band_workflow_id = ge.register_workflow(sentinel2_band_workflow)
    sentinel2_band_workflow_dataset_task = sentinel2_band_workflow_id.save_as_dataset(study_area, f"{s2_data_prefix}{b}", f"Sentinel-2 NRW area 10m {b}")
    # We start the download task and turn it into a future. This way we can await all tasks at once.
    download_tasks[b] = sentinel2_band_workflow_dataset_task.as_future(print_status=False, request_interval=60)

# the asyncio.gather function awaits all tasks at once and returns the final status as a list
download_task_results = await asyncio.gather(*download_tasks.values())

download_task_results

In [ ]:
## The datasets have been downloaded and have been registered with Geo Engine. We can now use them in a workflow.

## Either convert the list of download results into a dictionary with the band name as key:

# band_dataset_names = {band: task_result.info['dataset'] for band, task_result in zip(download_tasks.keys(), download_task_results)}
# band_dataset_names

## Or just use the dataset names as defined in the download step:

band_dataset_names = {
 'B02': user_id + ':y_sentinel2_nrw_crop_10m_B02',
 'B03': user_id + ':y_sentinel2_nrw_crop_10m_B03',
 'B04': user_id + ':y_sentinel2_nrw_crop_10m_B04',
 'B08': user_id + ':y_sentinel2_nrw_crop_10m_B08',
 'SCL': user_id + ':y_sentinel2_nrw_crop_10m_SCL'
}

band_dataset_names

## 3. Monthly, cloud-free aggregations of the Sentinel-2 bands & NDVI

For the training, we use the Sentinel-2 data of the bands 02, 03, 04, and 08. The scene classification layer (SCL) is used to filter out cloudy pixels for each band using an expression. The NDVI is calculated using an expression on band 4 and 8. 


Now for each variable we want to use in the training, we create a workflow that aggregates the data to monthly, cloud-free means.
The workflow is created using the `s2_cloud_free_aggregated_band_custom_input` method that can create the workflow for all the steps: loading data, removing clouded pixels, and aggregating them over the temporal domain.
This method takes the `DatasetName` of the band of interest and the name of the scene classification layer (SCL) as input.

It creates a workflow uses the [`GdalSource`](https://docs.geoengine.io/operators/gdalsource.html) to load both rasters and an [`Expression`](https://docs.geoengine.io/operators/expression.html) to filter out pixels that are marked "cloudy" in the SCL.
The, now cloud-free, data is aggregated to monthly means using the [`TemporalRasterAggregation`](https://docs.geoengine.io/operators/temporalrasteraggregation.html) operator. You can also build that workflow using the operators directly.

The NDVI is calculated using an expression on band 4 and 8.
For convenience, there is a `s2_cloud_free_aggregated_ndvi_custom_input` method that takes the `DatasetName` of the bands 4, 8, and the SCL as input.
It creates a workflow that loads the data, removes cloudy pixels and calculates the NDVI using an expression and then aggregates the data to monthly means.

In [ ]:
monthly_cloud_free_workflows = {}

for b in ["B02", "B03", "B04", "B08"]:
    band_dataset_name = band_dataset_names[b]
    scl_dataset_name = band_dataset_names["SCL"]
    sentinel2_band_workflow = ge.workflow_builder.blueprints.s2_cloud_free_aggregated_band_custom_input(band_id=band_dataset_name, scl_id=scl_dataset_name, granularity="months", window_size=1, aggregation_type="mean")
    monthly_cloud_free_workflows[b] = sentinel2_band_workflow

ndvi_workflow = ge.workflow_builder.blueprints.s2_cloud_free_aggregated_ndvi_custom_input(nir_dataset=band_dataset_names["B08"], red_dataset=band_dataset_names["B04"], scl_dataset=band_dataset_names["SCL"], granularity="months", window_size=1, aggregation_type="mean")
monthly_cloud_free_workflows["NDVI"] = ndvi_workflow

monthly_cloud_free_workflows

For the analysis of crops on fields, the temporal information is very important.
Lots of information can be gained by looking at the development of the NDVI over time.
Therefore, we want to generate for each point of interest the NDVI and other band information of the previous 8 months.
To do this, we wrap the workflow in a [`TimeShift](https://docs.geoengine.io/operators/timeshift.html) operator.
This way, we can use the existing workflows and just shift the temporal domain for all the months we are interested in.

In [ ]:
monthly_cloud_free_workflows_shifted = {}

for month_shift in range(0,-9, -1):
    monthly_cloud_free_workflows_shifted[month_shift] = {b: ge.workflow_builder.operators.TimeShift(granularity="months", value=month_shift, shift_type="relative", source=x) for b, x in monthly_cloud_free_workflows.items()}
    
monthly_cloud_free_workflows_shifted


## 4. Join the monthly Sentinel-2 data to the points of interest

This step combines the monthly aggregated Sentinel-2 data with the points of interest. The resulting dataset is then queried from directly from python and stored as a pandas dataframe.

Upload the points of interest to the Geo Engine. First we use GeoPandas to load the points of interest into a DataFrame. Then we use the `upload_dataframe` method to upload the points to the Geo Engine.

In [ ]:
points_df = gpd.read_file("group_sample_frac1_inspireId_use_utm32n.gpkg")
points_dataset_name = ge.upload_dataframe(points_df, "group_sample_frac1_inspireId")
points_dataset_name

To use the points in a Geo Engine workflow, we define a [`OgrSource`](https://docs.geoengine.io/operators/ogrsource.html) operator and pass the id of the uploaded points to it.

In [ ]:
points_source_operator = ge.workflow_builder.operators.OgrSource(points_dataset_name)
points_source_operator.to_workflow_dict()

Now, we create a workflow that uses [`RasterVectorJoin`](https://docs.geoengine.io/operators/rastervectorjoin.html) operator to join Sentinel-2 data to the points. This operator creates a new column for each band and adds the value of the raster pixel that is closest to each point. The points are provided as input by the [`OgrSource`](https://docs.geoengine.io/operators/ogrsource.html). Additionally the [`RasterVectorJoin`](https://docs.geoengine.io/operators/rastervectorjoin.html) takes up to 8 raster inputs. Here we use the already defined workflows that provide the monthly, cloud-free means of the Sentinel-2 bands and NDVI. 






In [ ]:
points_with_s2_cloud_free_shift = points_source_operator

for month_shift, month_bands in monthly_cloud_free_workflows_shifted.items():
    points_with_s2_cloud_free_shift = ge.workflow_builder.operators.RasterVectorJoin(
        raster_sources=[x for x in month_bands.values()],
        vector_source=points_with_s2_cloud_free_shift, #projected_points,
        new_column_names=[f"{b}_{month_shift}" for b in month_bands.keys()]
    )

       
points_with_s2_cloud_free_shift.to_workflow_dict()

Now we can register the workflow at the Geo Engine and execute it:

In [ ]:
workflow = ge.register_workflow(points_with_s2_cloud_free_shift)
workflow

The `ResultDescriptor` of the workflow is a `VectorResultDescriptor`.
It includes the description of all the columns that are created by the workflow.

In [ ]:
workflow.get_result_descriptor()

To query the workflow we need datetime objects for the start and end of the time interval we are interested in.
Since the `TimeShift` operator takes care to generate 8 previous months, we need only need to query the last month of the time interval.

In [ ]:
start_dt = datetime(2021, 10, 1, 0, 0, 0)
end_dt = start_dt

start_dt, end_dt

Now, we query the workflow that attaches the Sentinel-2 data for the area of interest.
We use a resolution of 10m, which is the native resolution if the Sentinel-2 bands.
The workflow result is transformed into a pandas dataframe automatically.

In [ ]:
gp_res = await workflow.vector_stream_into_geopandas(
    ge.QueryRectangle(
        spatial_bounds=ge.BoundingBox2D(
            xmin=xmin,
            ymin=ymin,
            xmax=xmax,
            ymax=ymax,
        ),
        time_interval=ge.TimeInterval(
            start=start_dt,
            end=end_dt,
        ),
        resolution=ge.SpatialResolution(
            10.0,
            10.0,
        ),
        srs="EPSG:32632",
))

# gp_res.to_file("gp_res_10_frac1_monthly_use_utm32n_multi_steps_12months_shift.gpkg", driver="GPKG")
gp_res

## 5. Train a Random-Forest on the monthly Sentinel-2 data and the NRW crop data

Now we can train a Random-Forest on the monthly Sentinel-2 data and the NRW crop data. We use the `sklearn` package for this. But first, we need to prepare the data.

First, we need to replace the nan values in the dataframe with a number that is not part of the dataset. For this example we use 0.
This is necessary since the sklearn RF does not support nan values.

In [ ]:
gp_train_1=gp_res.replace(np.nan, 0)
gp_train_1

Now we create the training input.
This also makes sure that the data is in the correct order when we train the RF.

In [ ]:
train_variable_order = ['B02_0', 'B02_-1', 'B02_-2', 'B02_-3', 'B02_-4',  'B02_-5',  'B02_-6', 'B02_-7', 'B02_-8',
        'B03_0', 'B03_-1', 'B03_-2', 'B03_-3', 'B03_-4',  'B03_-5',  'B03_-6', 'B03_-7', 'B03_-8',
        'B04_0', 'B04_-1', 'B04_-2', 'B04_-3', 'B04_-4',  'B04_-5',  'B04_-6', 'B04_-7', 'B04_-8',
        'B08_0', 'B08_-1', 'B08_-2', 'B08_-3', 'B08_-4',  'B08_-5',  'B08_-6', 'B08_-7', 'B08_-8',
        'NDVI_0', 'NDVI_-1', 'NDVI_-2', 'NDVI_-3', 'NDVI_-4',  'NDVI_-5',  'NDVI_-6', 'NDVI_-7', 'NDVI_-8']

x_list = gp_train_1[train_variable_order].values

The labels are stored in the column `USE_CODE`. 

In [ ]:
y_list = gp_train_1['USE_CODE'].replace(0, 'None')
y_list

First, we create a train-test split. We use 80% of the data for training and 20% for testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_list, y_list, test_size=0.2, random_state=31337, stratify=y_list)

Then, we train the RF on the training data:

In [ ]:
X = X_train
Y = y_train
clf = RandomForestClassifier(random_state=1337, class_weight='balanced_subsample', n_estimators=300)
clf = clf.fit(X, Y)
str(clf)

We can then create a report of the trained RF:

In [ ]:
x_test_predictions = clf.predict(X_test)

print(classification_report(y_test, x_test_predictions, labels=clf.classes_, zero_division=0, digits=3))

Print the confusion matrix:

In [ ]:
cm = confusion_matrix(y_test, clf.predict(X_test), labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot();

The confusion matrix shows that the class distribution is very imbalanced.
This is why the accuracy is so high.
The model is very good at predicting the most common classes.
This is also visible in the precision and recall scores in the classification report.
The precision and recall scores are very high for the most common classes and low for the less common classes. 
However the F1 score is still high.
To imporve the model, we can train on a larger dataset that contains more samples of the less common classes.
For this example, we are fine with using the trained RF as is.

## 6. Apply the trained Random-Forest to the monthly Sentinel-2 data

To generate a raster map of the field-use classes, we can apply the trained RF to the Sentinel-2 data.
For this we can create a workflow that generates the same monthly Sentinel-2 data as before, but this time want to apply the trained RF to all the pixels.
To do this, we query raster tiles from the Geo Engine and apply the RF to all pixels in each raster tile.
The result is a list of xarray data arrays.
Normally we would retrain the model on all data, but for demonstration and simplicity we use the model trained on the train/test split.

Now we specify the query rectangle we are interested in. It is a subeset of the area of interest defined in step 2.

In [ ]:
start_dt = datetime(2021, 10, 15, 0, 0, 0)
end_dt = datetime(2021, 10, 15, 0, 0, 0)

box_size = 512

box_center_x = 0.5 * (xmin + xmax)
box_center_y = 0.5 * (ymin + ymax)

box_x_min = box_center_x - box_size *10
box_x_max = box_center_x + box_size *10
box_y_min = box_center_y - box_size *10
box_y_max = box_center_y + box_size *10

query_rect = ge.QueryRectangle(
        spatial_bounds=ge.BoundingBox2D(
            xmin=box_x_min,
            ymin=box_y_min,
            xmax=box_x_max,
            ymax=box_y_max,
        ),
        time_interval=ge.TimeInterval(
            start=start_dt,
            end=end_dt,
        ),
        resolution=ge.SpatialResolution(
            10.0,
            10.0,
        ),
        srs="EPSG:32632",
)

query_rect

We also need a map from category to number. This is necessary since the RF produces class names and we want to store the result as a raster dataset.

In [ ]:
code_id_map = {
    'None': 0,
    'AF': 1,
    'DA': 2,
    'EP': 3,
    'EW': 4,
    'GL': 5,
    'GM': 6,
    'GT': 7,
    'HF': 8,
    'HP': 9,
    'OE': 10,
    'PA': 11,
    'SF': 12,
    'SL': 13,
    'ZP': 14,    
}

id_code_map = {v: k for k, v in code_id_map.items()}

Now we query all the workflows in parallel and zip the results together. The result is a list of tiles, where each tile contains the monthly Sentinel-2 data for one band and month.
For each tile (list) we apply the RF to the data and store the result as a new tile. The result is a list of tiles, where each tile contains the predicted crop class for one pixel.

In [ ]:
query_keys = train_variable_order
queries = [
    ge.register_workflow(monthly_cloud_free_workflows_shifted[int(b.split("_")[1])][b.split("_")[0]]).raster_stream(query_rect) for b in query_keys
]

res_arrays = []

async for tile_stac in zip_longest(*queries):
    tiles_as_xarrays = [tile.to_xarray() for tile in tile_stac]

    arr_stack = xr.concat(tiles_as_xarrays, dim="band")
    arr_stack_2 = arr_stack.transpose("y", "x", "band")

    rf_input = arr_stack_2.values.reshape((box_size * box_size, len(tile_stac)))
  
    np.nan_to_num(rf_input, copy=False, nan=0, posinf=None, neginf=None)

    pred_classes = clf.predict(rf_input)
    pred_numbers = np.vectorize(code_id_map.get)(pred_classes)

    res_array = pred_numbers.reshape((box_size, box_size))
    
    da = xr.DataArray(
        data=res_array,
        dims=["y", "x"],
        coords=arr_stack_2.coords,
        attrs=dict(
            description="Predicted use."
        ),
    )      

    res_arrays.append(da)

We can now print the result as a image:

In [ ]:
for arr in res_arrays:
    plt.figure()
    ax = arr.plot(levels=code_id_map.values(), cmap="tab20")
    

In the classification map, we can see that the model is able to generate sound field shapes.
It is also visible that the model is able to distinguish between the different field-use classes.
When comparing the classification map to the NRW crop data, we can see that the model is able to predict the field-use classes for most of the fields.
There are some areas, mainly at the south border of the map, where the model predicted "Greenland (GL)".
This area is not covered by a field and therefore not labeled in the NRW crop data.
However, those areas are Greenland in OpenStreetMap and therefore the model predicted "Greenland (GL)" might be correct.


And finally, we can store the result as a new GeoTiFF file:

In [ ]:
for i, arr in enumerate(res_arrays):
    arr.rio.to_raster(
    f"arr_{i}.tif",
    tiled=True,  # GDAL: By default striped TIFF files are created. This option can be used to force creation of tiled TIFF files.
    windowed=True,  # rioxarray: read & write one window at a time
)